# Task 1: Link Prediction

Desription: In the codes below, I first created a subgraph based on random-sampled nodes from network file (sample node size = 2000). Then I add the nodes from network to make the randomly-selected subgraph can be more representative of the whole network. After the graph is constructed, I computed the distance among each nodes based on Jaccard Coefficients, and finally output the prediction results to a csv file. 

In [17]:
import pandas as pd
import numpy as np
from math import log
import networkx as nx
import time

In [7]:
network = pd.read_csv('network.tsv',sep = '\t',header=None)
network = network.rename(columns={0: 'node_id1', 1: 'node_id2'})
network['link'] = 1

In [8]:
network.head()

,node_id1,node_id2,link
0,3942361,4009630,1
1,1862789,2403557,1
2,3559086,3449838,1
3,4268588,6041523,1
4,3067063,3845402,1


In [9]:
network['edges'] = list(zip(network['node_id1'], network['node_id2']))
network.head()    

,node_id1,node_id2,link,edges
0,3942361,4009630,1,"(3942361, 4009630)"
1,1862789,2403557,1,"(1862789, 2403557)"
2,3559086,3449838,1,"(3559086, 3449838)"
3,4268588,6041523,1,"(4268588, 6041523)"
4,3067063,3845402,1,"(3067063, 3845402)"


Randomly select nodes from network file (node number = 2000), and build graph.

In [10]:
# get the unique nodes
nodes = []
nodes += network.node_id1.unique().tolist()
nodes += network.node_id2.unique().tolist()

# define a random set of nodes S
import random
sample_nodes = random.sample(nodes, 2000)
sample_df = network[(network.node_id1.isin(sample_nodes) == True) | (network.node_id2.isin(sample_nodes) == True)]

In [11]:
# construct graph
sample_edge_list = sample_df['edges'].tolist()
G = nx.Graph()
G.add_edges_from(sample_edge_list)

In [12]:
sample_df2 = network[(network.node_id1.isin(list(G.nodes())) == True) & (network.node_id2.isin(list(G.nodes())) == True)]

sample_edge_list2 = sample_df2['edges'].tolist()
G.add_edges_from(sample_edge_list2)

In [23]:
start_time = time.time()

preds = nx.jaccard_coefficient(G, list(G.edges()))
# preds = nx.preferential_attachment(G, list(G.edges()))
# preds = nx.adamic_adar_index(G, list(G.edges()))
# preds = nx.resource_allocation_index(G, list(G.edges()))


dct = {}
for u, v, p in preds:
    dct[(u,v)] = p

# edge list (in descending order)
results = [item[0] for item in sorted(dct.items(),key=lambda x:x[1], reverse = True)]  

end_time = time.time()
print("Process took {:.04f} seconds".format(end_time - start_time))

Process took 7.2603 seconds


In [19]:
df = pd.DataFrame(results, columns = ['vertex1', 'vertex2'])
top_df = df.head(50000)

In [20]:
top_df.to_csv('task1_result(Jaccard).csv', index = False, encoding = 'UTF-8')
# top_df.to_csv('task1_result(preferential_attachment).csv', index = False, encoding = 'UTF-8')
# top_df.to_csv('task1_result(adamic_adar_index).csv', index = False, encoding = 'UTF-8')
# top_df.to_csv('task1_result(resource_allocation_index).csv', index = False, encoding = 'UTF-8')